In [1]:
import tensorflow as tf
import time, os
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.preprocessing import LabelEncoder

In [2]:
class CharNetConfig(object):
    def __init__(self, params=None):
        if params is None:
            self.conv_layers = [
                    [256, 7, 3],
                    [256, 7, 3],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, None],
                    [256, 3, 3]
                ]
            self.fc_layers = [1024, 512, 10]
            self.l0 = 128
            self.alstr = 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’’’/\|_@#$%ˆ&* ̃‘+-=<>()[]{}]'
            self.alphabet_size = len(self.alstr)
        else:
            self.conv_layers = params['conv_layers']
            self.fc_layers = params['fc_layers']
            self.l0 = params['l0']
            slef.alstr = params['alstr']

In [3]:
class CharNet(object):
    """docstring for CharNet."""
    def __init__(self, conv_layers,
                        fc_layers,
                        l0,
                        alphabet_size,
                        encoder,
                        **args
    ):
        super(CharNet, self).__init__()
        tf.set_random_seed(time.time())
        self.l0 = l0
        self.conv_layers = conv_layers
        self.fc_layers = fc_layers
        self.alphabet_size = alphabet_size

        initializer = tf.contrib.layers.xavier_initializer()

        with tf.name_scope('Input'):
            self.input_x = tf.placeholder(tf.int64, shape=[None, self.l0],
                                          name='input_x')
            self.input_y = tf.placeholder(tf.float32, shape=[None, 6],
                                          name='input_y')
            self.dropout_keep_prob = tf.placeholder(tf.float32,
                                                    name='dropout_keep_prob')

        with tf.name_scope('Embedding'):
            x = tf.nn.embedding_lookup(encoder, self.input_x)
            x = tf.expand_dims(x, -1)

        # Configure conv layers
        for i, layer_params in enumerate(conv_layers):
            with tf.name_scope("Convolution"):
                filter_param = [
                    layer_params[1],
                    x.get_shape()[2].value, # l0
                    x.get_shape()[3].value, # channels
                    layer_params[0]
                ]
                W = tf.Variable(initializer(filter_param), dtype='float32', name='filter')

                conv_layer = tf.nn.conv2d(x, W, [1, 1, 1, 1], 'VALID', name='conv')
                conv_layer = tf.nn.relu(conv_layer, name='act_relu')

            if not layer_params[-1] is None:
                with tf.name_scope("MaxPooling"):
                    pool_layer = tf.nn.max_pool(conv_layer,
                                            ksize=[1, layer_params[-1], 1, 1],
                                            strides=[1, layer_params[-1], 1, 1],
                                            padding='VALID')
                    x = tf.transpose(pool_layer, [0, 1, 3, 2])
            else:
                x = tf.transpose(conv_layer, [0, 1, 3, 2])

        # flatten conv output for fc
        with tf.name_scope("Flatten"):
            x = tf.contrib.layers.flatten(x)

        # Configure fc layers
        for i, layer_units in enumerate(fc_layers):
            with tf.name_scope("FullyConnected"):
                W = tf.Variable(initializer([x.get_shape()[-1].value, layer_units]),
                                dtype='float32', name='W')
                b = tf.Variable(initializer([layer_units]),
                                dtype='float32', name='W')
                x = tf.nn.xw_plus_b(x, W, b, name='fully-connected')
                x = tf.nn.relu(x)

            with tf.name_scope("Dropout"):
                x = tf.nn.dropout(x, self.dropout_keep_prob)

        with tf.name_scope("Output"):
            W = tf.Variable(initializer([x.get_shape()[-1].value, 6]),
                            dtype='float32', name='W')
            b = tf.Variable(initializer([6]),
                            dtype='float32', name='W')
            self.yhat = tf.nn.sigmoid(tf.matmul(x, W) + b, name='output')

        with tf.name_scope("Loss"):
            self.loss = tf.losses.log_loss(self.input_y, self.yhat)

In [4]:
class Data(object):
    def __init__(self, file_path, alstr, l0, is_dev=False, batch_size=128, **args):
        self.alstr = alstr
        self.l0 = l0
        self.is_dev = is_dev
        self.batch_size = batch_size
        self.raw_data = pd.read_csv(file_path)

        self.alphabet = self.make_alphabet(self.alstr)
        self.encoder, self.e_dict = self.one_hot_encoder(self.alphabet)
        self.alphabet_size = len(self.alphabet)
    
        self.input_x = self.process_full_description(self.raw_data)
        if not self.is_dev:
            self.y = self.generate_y(self.raw_data)
            
    def shuffling(self):
        shuffle_indices = np.random.permutation(np.arange(len(self.input_x)))
        self.input_x = self.input_x[shuffle_indices]
        self.y = self.y[shuffle_indices]

    def next_batch(self, batch_num):
        data_size = len(self.input_x)
        start = batch_num * self.batch_size
        end = min((batch_num + 1) * self.batch_size, data_size)
        batch_x = self.input_x[start:end]
        if self.is_dev == False:
            batch_y = self.y[start:end]
        else:
            batch_y = None
        return batch_x, batch_y

    def process_full_description(self, df):
        df['comment_text'] = df['comment_text'].astype('str')
        df['desc_vecs'] = df['comment_text'].apply(
                lambda x: self.doc_process(x, self.e_dict)
        )
        return df['desc_vecs'].values

    def generate_y(self, df):
        list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
        y = df[list_classes].values
        return y

    def one_hot_encoder(self, alphabet):
        encoder_dict = {}
        encoder = []

        encoder_dict['UNK'] = 0
        encoder.append(np.zeros(len(alphabet), dtype='float32'))

        for i, alpha in enumerate(alphabet):
            onehot = np.zeros(len(alphabet), dtype='float32')
            encoder_dict[alpha] = i + 1
            onehot[i] = 1
            encoder.append(onehot)

        encoder = np.array(encoder, dtype='float32')
        return encoder, encoder_dict

    def doc_process(self, desc, e_dict):
        l = self.l0
        desc = desc.strip().lower()
        min_len = min(l, len(desc))
        doc_vec = np.zeros(l, dtype='int64')
        for j in range(min_len):
            if desc[j] in e_dict:
                doc_vec[j] = e_dict[desc[j]]
            else:
                doc_vec[j] = e_dict['UNK']
        return doc_vec

    def make_alphabet(self, alstr):
        return [char for char in alstr]

In [5]:
train_file = '../inputs/train.csv'
dev_file = '../inputs/test.csv'
config = CharNetConfig()

In [6]:
train_data = Data(train_file, config.alstr, config.l0, is_dev=False, batch_size=128)
dev_data = Data(dev_file, config.alstr, config.l0, is_dev=True, batch_size=128)
conf = tf.ConfigProto()
conf.gpu_options.allow_growth=True

In [7]:
with tf.Session(config=conf) as sess:
    charnet = CharNet(config.conv_layers, config.fc_layers, config.l0, config.alphabet_size, train_data.encoder)

    optimizer = tf.train.AdamOptimizer()
    grads = optimizer.compute_gradients(charnet.loss)
    train_op = optimizer.apply_gradients(grads)

    timestamp = str(int(time.time()))
    out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    print("Writing to {}\n".format(out_dir))

    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", charnet.loss)

    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary])
    train_summary_dir = os.path.join(out_dir, "summaries", "train")
    train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

    # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
    checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
    checkpoint_prefix = os.path.join(checkpoint_dir, "model")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    saver = tf.train.Saver(tf.global_variables())

    sess.run(tf.global_variables_initializer())

    def train_step(x_batch, y_batch, step):
        """
        A single training step
        """
        y_batch = np.reshape(y_batch, (-1, 6))
        mm = x_batch.tolist()
        x_batch = np.array([k.tolist() for k in mm])

        feed_dict = {
            charnet.input_x: x_batch,
            charnet.input_y: y_batch,
            charnet.dropout_keep_prob: .5
        }

        _, summaries, loss = sess.run(
            [train_op,
             train_summary_op,
             charnet.loss],
            feed_dict
        )

        print("step {}, loss {:g}".format(step, loss))
        train_summary_writer.add_summary(summaries, step)

    def predict_on_test(x_batch, results):
        mm = x_batch.tolist()
        x_batch = np.array([k.tolist() for k in mm])

        feed_dict = {
            charnet.input_x: x_batch,
            charnet.dropout_keep_prob: 1.0
        }
        result = sess.run([charnet.yhat], feed_dict)
        results.append(result)

    for epoch in range(50):
        print("epoch is: {}".format(epoch))
        train_data.shuffling()
        for i in range(int(len(train_data.y)/train_data.batch_size) + 1):
            input_x, y = train_data.next_batch(i)
            train_step(input_x, y, i)

            if i % 1000 == 0:
                path = saver.save(sess, './model.ckpt')
                print("Epoch {}, Saved model checkpoint to {}\n".format(epoch, path))
    
    results = []
    submission = dev_data.raw_data['id']
    for i in range(int(len(dev_data.input_x)/dev_data.batch_size) + 1):
        input_x, _ = dev_data.next_batch(i)
        predict_on_test(input_x, results)
    
    preds = []
    for result in results:
        for elem in result[0]:
            preds.append(elem)

    preds = np.array(preds)
#     list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
#     submission = pd.DataFrame(preds, columns=list_classes)
#     submission['id'] = dev_data.raw_data['id']
#     cols = submission.columns.tolist()
#     cols = cols[-1:] + cols[:-1]
#     submission = submission[cols]
#     submission.to_csv('../outputs/submission.csv', index=False)

Writing to /home/arac/aramac/ToxicComment/src/runs/1515132886

epoch is: 0
step 0, loss 0.79395
Epoch 0, Saved model checkpoint to ./model.ckpt

step 1, loss 0.770838
step 2, loss 0.756137
step 3, loss 0.753829
step 4, loss 0.756525
step 5, loss 0.755891
step 6, loss 0.752323
step 7, loss 0.751671
step 8, loss 0.744443
step 9, loss 0.744474
step 10, loss 0.742956
step 11, loss 0.743436
step 12, loss 0.749773
step 13, loss 0.751246
step 14, loss 0.746516
step 15, loss 0.745767
step 16, loss 0.741119
step 17, loss 0.742167
step 18, loss 0.74344
step 19, loss 0.737117
step 20, loss 0.73583
step 21, loss 0.736527
step 22, loss 0.737413
step 23, loss 0.737428
step 24, loss 0.734127
step 25, loss 0.733267
step 26, loss 0.733422
step 27, loss 0.727863
step 28, loss 0.72638
step 29, loss 0.727809
step 30, loss 0.735759
step 31, loss 0.732432
step 32, loss 0.731386
step 33, loss 0.724961
step 34, loss 0.716178
step 35, loss 0.723224
step 36, loss 0.722291
step 37, loss 0.717779
step 38, loss 0.

step 351, loss 0.302921
step 352, loss 0.248709
step 353, loss 0.240466
step 354, loss 0.301975
step 355, loss 0.308153
step 356, loss 0.294461
step 357, loss 0.320211
step 358, loss 0.28394
step 359, loss 0.25174
step 360, loss 0.338917
step 361, loss 0.264354
step 362, loss 0.276579
step 363, loss 0.241012
step 364, loss 0.228652
step 365, loss 0.350224
step 366, loss 0.294358
step 367, loss 0.27916
step 368, loss 0.278806
step 369, loss 0.305246
step 370, loss 0.270626
step 371, loss 0.26422
step 372, loss 0.261171
step 373, loss 0.236797
step 374, loss 0.256245
step 375, loss 0.330323
step 376, loss 0.22643
step 377, loss 0.25732
step 378, loss 0.288394
step 379, loss 0.220867
step 380, loss 0.231642
step 381, loss 0.224192
step 382, loss 0.305513
step 383, loss 0.296374
step 384, loss 0.307143
step 385, loss 0.262516
step 386, loss 0.315249
step 387, loss 0.268249
step 388, loss 0.222385
step 389, loss 0.300475
step 390, loss 0.232203
step 391, loss 0.286564
step 392, loss 0.24333

step 705, loss 0.13877
step 706, loss 0.153147
step 707, loss 0.131975
step 708, loss 0.156596
step 709, loss 0.190148
step 710, loss 0.207457
step 711, loss 0.116428
step 712, loss 0.156891
step 713, loss 0.164158
step 714, loss 0.191949
step 715, loss 0.174258
step 716, loss 0.183468
step 717, loss 0.148524
step 718, loss 0.163969
step 719, loss 0.133828
step 720, loss 0.201386
step 721, loss 0.109701
step 722, loss 0.123219
step 723, loss 0.158567
step 724, loss 0.122894
step 725, loss 0.260986
step 726, loss 0.150117
step 727, loss 0.157176
step 728, loss 0.149717
step 729, loss 0.141307
step 730, loss 0.103659
step 731, loss 0.171613
step 732, loss 0.200991
step 733, loss 0.134622
step 734, loss 0.160339
step 735, loss 0.150276
step 736, loss 0.111821
step 737, loss 0.134182
step 738, loss 0.165559
step 739, loss 0.137774
step 740, loss 0.189557
step 741, loss 0.155887
step 742, loss 0.186028
step 743, loss 0.0986591
step 744, loss 0.14034
step 745, loss 0.126779
step 746, loss 0.

step 305, loss 0.0906817
step 306, loss 0.129655
step 307, loss 0.12794
step 308, loss 0.138041
step 309, loss 0.100928
step 310, loss 0.0780866
step 311, loss 0.214482
step 312, loss 0.113797
step 313, loss 0.102416
step 314, loss 0.144346
step 315, loss 0.124427
step 316, loss 0.10944
step 317, loss 0.139586
step 318, loss 0.0948863
step 319, loss 0.105617
step 320, loss 0.11829
step 321, loss 0.0992259
step 322, loss 0.129041
step 323, loss 0.174528
step 324, loss 0.109396
step 325, loss 0.205699
step 326, loss 0.152622
step 327, loss 0.119564
step 328, loss 0.103551
step 329, loss 0.105546
step 330, loss 0.0937934
step 331, loss 0.0955085
step 332, loss 0.196213
step 333, loss 0.185803
step 334, loss 0.109199
step 335, loss 0.0868108
step 336, loss 0.127494
step 337, loss 0.198884
step 338, loss 0.108476
step 339, loss 0.107321
step 340, loss 0.15459
step 341, loss 0.117662
step 342, loss 0.0819726
step 343, loss 0.18098
step 344, loss 0.10207
step 345, loss 0.106529
step 346, loss

step 656, loss 0.107773
step 657, loss 0.095159
step 658, loss 0.111021
step 659, loss 0.164357
step 660, loss 0.174013
step 661, loss 0.105177
step 662, loss 0.128479
step 663, loss 0.185145
step 664, loss 0.141118
step 665, loss 0.187246
step 666, loss 0.117544
step 667, loss 0.0964047
step 668, loss 0.0867307
step 669, loss 0.124561
step 670, loss 0.154618
step 671, loss 0.0996463
step 672, loss 0.160748
step 673, loss 0.0916121
step 674, loss 0.104619
step 675, loss 0.1089
step 676, loss 0.0941086
step 677, loss 0.109717
step 678, loss 0.0871601
step 679, loss 0.133245
step 680, loss 0.164307
step 681, loss 0.106439
step 682, loss 0.114696
step 683, loss 0.097244
step 684, loss 0.136957
step 685, loss 0.0974788
step 686, loss 0.143668
step 687, loss 0.140833
step 688, loss 0.110761
step 689, loss 0.155743
step 690, loss 0.160974
step 691, loss 0.116758
step 692, loss 0.120186
step 693, loss 0.119692
step 694, loss 0.132625
step 695, loss 0.196737
step 696, loss 0.102416
step 697, l

...

step 596, loss 0.028175
step 597, loss 0.0238563
step 598, loss 0.0396841
step 599, loss 0.0372151
step 600, loss 0.0260296
step 601, loss 0.0572899
step 602, loss 0.0475885
step 603, loss 0.0590413
step 604, loss 0.0649317
step 605, loss 0.0480678
step 606, loss 0.0442504
step 607, loss 0.0387728
step 608, loss 0.067842
step 609, loss 0.0271748
step 610, loss 0.0296773
step 611, loss 0.0353673
step 612, loss 0.0368513
step 613, loss 0.0551451
step 614, loss 0.0369077
step 615, loss 0.0345866
step 616, loss 0.0559626
step 617, loss 0.0596052
step 618, loss 0.0274583
step 619, loss 0.0371766
step 620, loss 0.0534045
step 621, loss 0.0645791
step 622, loss 0.0362081
step 623, loss 0.0422697
step 624, loss 0.0469442
step 625, loss 0.046304
step 626, loss 0.0422565
step 627, loss 0.0507372
step 628, loss 0.0396821
step 629, loss 0.0451369
step 630, loss 0.0437972
step 631, loss 0.0426
step 632, loss 0.0574868
step 633, loss 0.0397334
step 634, loss 0.0797309
step 635, loss 0.0270174
step 6

step 191, loss 0.0355026
step 192, loss 0.0531414
step 193, loss 0.0268859
step 194, loss 0.0234217
step 195, loss 0.0676583
step 196, loss 0.0437127
step 197, loss 0.0710988
step 198, loss 0.0705673
step 199, loss 0.0543861
step 200, loss 0.0313755
step 201, loss 0.0449345
step 202, loss 0.0327928
step 203, loss 0.0590296
step 204, loss 0.0606912
step 205, loss 0.0836924
step 206, loss 0.030617
step 207, loss 0.0300501
step 208, loss 0.0403094
step 209, loss 0.0387316
step 210, loss 0.0429929
step 211, loss 0.0512533
step 212, loss 0.019212
step 213, loss 0.043369
step 214, loss 0.0480871
step 215, loss 0.0419051
step 216, loss 0.0462729
step 217, loss 0.0272592
step 218, loss 0.0298454
step 219, loss 0.0334778
step 220, loss 0.0611452
step 221, loss 0.0537756
step 222, loss 0.0322301
step 223, loss 0.0386035
step 224, loss 0.033413
step 225, loss 0.0214513
step 226, loss 0.0466455
step 227, loss 0.0520667
step 228, loss 0.0421045
step 229, loss 0.0413046
step 230, loss 0.0505779
step

step 527, loss 0.0451153
step 528, loss 0.0208006
step 529, loss 0.0523349
step 530, loss 0.0437089
step 531, loss 0.0623073
step 532, loss 0.0377658
step 533, loss 0.0221561
step 534, loss 0.0567266
step 535, loss 0.0408476
step 536, loss 0.0244844
step 537, loss 0.0515925
step 538, loss 0.0339369
step 539, loss 0.0322642
step 540, loss 0.0441639
step 541, loss 0.0394828
step 542, loss 0.0423449
step 543, loss 0.042568
step 544, loss 0.0466562
step 545, loss 0.0339177
step 546, loss 0.0549377
step 547, loss 0.0414305
step 548, loss 0.0392084
step 549, loss 0.0740772
step 550, loss 0.0504743
step 551, loss 0.0450074
step 552, loss 0.0574774
step 553, loss 0.0267854
step 554, loss 0.0276603
step 555, loss 0.0210181
step 556, loss 0.0477928
step 557, loss 0.0652768
step 558, loss 0.0133089
step 559, loss 0.0191821
step 560, loss 0.039187
step 561, loss 0.0417292
step 562, loss 0.0369544
step 563, loss 0.0294775
step 564, loss 0.0318046
step 565, loss 0.0414385
step 566, loss 0.0633991
st

step 111, loss 0.0466097
step 112, loss 0.0291296
step 113, loss 0.0541501
step 114, loss 0.0186365
step 115, loss 0.0284163
step 116, loss 0.0787193
step 117, loss 0.0238348
step 118, loss 0.0621455
step 119, loss 0.0281386
step 120, loss 0.0256472
step 121, loss 0.0314714
step 122, loss 0.0665502
step 123, loss 0.0438398
step 124, loss 0.0255268
step 125, loss 0.0248557
step 126, loss 0.0504786
step 127, loss 0.0411085
step 128, loss 0.0396447
step 129, loss 0.0428617
step 130, loss 0.0300273
step 131, loss 0.051541
step 132, loss 0.0387542
step 133, loss 0.0289812
step 134, loss 0.0385253
step 135, loss 0.0363039
step 136, loss 0.0559439
step 137, loss 0.0563958
step 138, loss 0.0612802
step 139, loss 0.0284877
step 140, loss 0.0538944
step 141, loss 0.0711562
step 142, loss 0.0343294
step 143, loss 0.0674525
step 144, loss 0.0515147
step 145, loss 0.0673337
step 146, loss 0.0550306
step 147, loss 0.033163
step 148, loss 0.0529567
step 149, loss 0.0386641
step 150, loss 0.0366194
st

step 447, loss 0.0513705
step 448, loss 0.0629186
step 449, loss 0.0400163
step 450, loss 0.046848
step 451, loss 0.0594863
step 452, loss 0.034164
step 453, loss 0.037375
step 454, loss 0.057852
step 455, loss 0.0491303
step 456, loss 0.044183
step 457, loss 0.0450162
step 458, loss 0.0516827
step 459, loss 0.0301022
step 460, loss 0.0801767
step 461, loss 0.039747
step 462, loss 0.0543294
step 463, loss 0.0240534
step 464, loss 0.0290741
step 465, loss 0.025296
step 466, loss 0.0483181
step 467, loss 0.0252374
step 468, loss 0.0363518
step 469, loss 0.0531
step 470, loss 0.0582622
step 471, loss 0.0505468
step 472, loss 0.101586
step 473, loss 0.0180262
step 474, loss 0.0456044
step 475, loss 0.0306257
step 476, loss 0.0373431
step 477, loss 0.0380414
step 478, loss 0.0443952
step 479, loss 0.0469009
step 480, loss 0.0372625
step 481, loss 0.0735667
step 482, loss 0.0533694
step 483, loss 0.0364906
step 484, loss 0.0448288
step 485, loss 0.0504924
step 486, loss 0.0359853
step 487, l

In [8]:
preds

array([[  2.27914562e-12,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  8.61136218e-10,   0.00000000e+00,   0.00000000e+00,
          1.88755696e-32,   0.00000000e+00,   0.00000000e+00],
       [  1.00003922e-13,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  5.03141027e-14,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.09421285e-15,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  4.92943674e-01,   8.20797250e-12,   1.00171501e-05,
          8.97780620e-03,   6.17013238e-02,   1.30446500e-03]], dtype=float32)

In [19]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

path = '../input/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE = '../inputs/glove.6B.100d.txt'
TRAIN_DATA_FILE = '../inputs/train.csv'
TEST_DATA_FILE = '../inputs/test.csv'

embed_size = 100 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use

train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [23]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(100, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_t, y, batch_size=512, epochs=5)
y_test = model.predict([X_te], batch_size=1024, verbose=1)

Epoch 1/5
95851/95851 [==============================] - 39s 405us/step - loss: 0.2559 - acc: 0.8910
Epoch 2/5
95851/95851 [==============================] - 38s 399us/step - loss: 0.0971 - acc: 0.9626
Epoch 3/5
95851/95851 [==============================] - 39s 404us/step - loss: 0.0757 - acc: 0.9709
Epoch 4/5
95851/95851 [==============================] - 38s 394us/step - loss: 0.0663 - acc: 0.9752
Epoch 5/5
226998/226998 [==============================] - 15s 64us/step


In [24]:
sample_submission = pd.read_csv('../inputs/sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('../outputs/submission.csv', index=False)

In [25]:
sample_submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,6044863,0.045828,1.442954e-05,1.578895e-04,7.285278e-07,7.439613e-04,2.057297e-06
1,6102620,0.000011,8.571185e-17,4.005610e-14,1.394462e-21,7.633865e-12,2.963135e-20
2,14563293,0.000015,2.356751e-16,9.898901e-14,6.060018e-21,1.549776e-11,1.118565e-19
3,21086297,0.003083,2.900108e-09,1.179229e-07,1.049499e-11,1.854192e-06,6.058015e-11
4,22982444,0.000309,2.199212e-12,2.561625e-10,1.119349e-15,1.087195e-08,1.073433e-14
5,24388733,0.000255,1.816373e-12,1.719728e-10,5.113075e-16,7.389388e-09,4.620039e-15
6,26195914,0.000196,5.863141e-13,7.679023e-11,1.479330e-16,4.432692e-09,1.468261e-15
7,31769073,0.006696,2.836649e-08,8.776565e-07,2.016696e-10,1.023830e-05,9.392888e-10
8,35289443,0.006979,2.980697e-08,9.262874e-07,2.459745e-10,1.138540e-05,1.121661e-09
9,38393350,0.000083,4.315636e-14,8.156861e-12,4.703612e-18,6.762179e-10,5.833123e-17
